# **自定義訓練流程 (Pytorch)**
此份程式碼為 Custom_training 的 PyTorch 參考寫法。
## 本章節內容大綱
* ### [建立資料集](#CreateDataset)
* ### [建構模型](#BuildModel)
* ### [訓練模型](#TrainModel)
* ### [評估模型](#EvaluateModel)
---

## 匯入套件

In [ ]:
!pip install torchsummary

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

# PyTorch 相關套件
import torch
from torchsummary import summary

<a name="CreateDataset"></a>
## 建立資料集

In [ ]:
# 上傳資料
!wget -q https://github.com/TA-aiacademy/course_3.0/releases/download/DL/Data_part4.zip
!unzip -q Data_part4.zip

In [ ]:
df = pd.read_csv('./Data/bodyperformance.csv')
df.head()

In [ ]:
X = torch.from_numpy(df.iloc[:, :-1].values)
y = torch.from_numpy(df['class'].values)

In [ ]:
y_onehot = torch.nn.functional.one_hot(y).double()

In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_valid, y_train, y_valid = train_test_split(X, y_onehot,
                                                      test_size=0.2,
                                                      random_state=17,
                                                      stratify=y)

In [ ]:
print(f'X_train shape: {X_train.shape}')
print(f'X_valid shape: {X_valid.shape}')
print(f'y_train shape: {y_train.shape}')
print(f'y_valid shape: {y_valid.shape}')

In [ ]:
# Feature scaling
from sklearn.preprocessing import StandardScaler, MinMaxScaler
sc = StandardScaler()
X_train = sc.fit_transform(X_train, y_train)
X_valid = sc.transform(X_valid)

In [ ]:
batch_size = 64

# 準備訓練資料集
train_dataset = torch.utils.data.TensorDataset(torch.Tensor(X_train),
                                               torch.Tensor(y_train))
train_dataset = torch.utils.data.DataLoader(train_dataset,
                                            batch_size=batch_size,
                                            shuffle=True)

# 準備驗證資料集
val_dataset = torch.utils.data.TensorDataset(torch.Tensor(X_valid),
                                             torch.Tensor(y_valid))
val_dataset = torch.utils.data.DataLoader(val_dataset,
                                          batch_size=batch_size)

<a name="BuildModel"></a>
## 建構模型

In [ ]:
def build_model(input_shape, output_shape):
    torch.cuda.empty_cache()
    torch.manual_seed(17)

    model = torch.nn.Sequential(
        torch.nn.Linear(input_shape, 32),
        torch.nn.SiLU(),
        torch.nn.Linear(32, 32),
        torch.nn.SiLU(),
        torch.nn.Linear(32, 32),
        torch.nn.SiLU(),
        torch.nn.Linear(32, output_shape))
    return model

In [ ]:
model = build_model(X_train[0].shape[0], y_onehot.shape[1])
summary(model, X_train[0].shape, device='cpu')

In [ ]:
# Instantiate an optimizer to train the model
loss_fn = torch.nn.CrossEntropyLoss()
# Instantiate a loss function
optimizer = torch.optim.NAdam(model.parameters(), lr=0.001)

<a name="TrainModel"></a>
## 訓練模型

In [ ]:
import time
import tqdm

# 將模型和損失函數放入 GPU 記憶體當中
if torch.cuda.is_available():
    model.cuda()
    loss_fn.cuda()

epochs = 10

# 創建 list 分別存放訓練集 acc, loss 和驗證集 acc
train_acc_list, train_loss_list = [], []
val_acc_list, val_loss_list = [], []

# 訓練的迭代過程
for epoch in range(epochs):
    start_time = time.time()
    t_bar = tqdm.tqdm_notebook(enumerate(train_dataset),
                               total=len(train_dataset),
                               desc=f'Epoch {epoch}')
    train_correct = 0
    train_samples = 0
    for step, (x_batch_train, y_batch_train) in t_bar:
        if torch.cuda.is_available():
            x_batch_train = x_batch_train.cuda()
            y_batch_train = y_batch_train.cuda()

        optimizer.zero_grad()
        outputs = model(x_batch_train)
        loss_value = loss_fn(outputs, y_batch_train)
        loss_value.backward()  # 計算參數上的梯度
        optimizer.step()  # 更新參數

        predict_cls = torch.argmax(outputs, 1)
        target_cls = torch.argmax(y_batch_train, 1)
        train_samples += target_cls.size(0)
        train_correct += (predict_cls == target_cls).sum().item()

    print('Training loss over epoch: %.4f' % (float(loss_value.item()),))
    train_acc = train_correct / train_samples  # 平均所有批次的評估結果
    print('Training acc over epoch: %.4f' % (float(train_acc),))

    # 將訓練的評估結果儲存下來
    train_acc_list.append(train_acc)
    train_loss_list.append(loss_value.cpu().detach().numpy())

    # 驗證集的迭代結果
    val_correct = 0
    val_samples = 0
    with torch.no_grad():
        for x_batch_val, y_batch_val in val_dataset:
            if torch.cuda.is_available():
                x_batch_val = x_batch_val.cuda()
                y_batch_val = y_batch_val.cuda()
            val_logits = model(x_batch_val)
            val_pred_cls = torch.argmax(val_logits, 1)
            val_target_cls = torch.argmax(y_batch_val, 1)
            val_samples += val_target_cls.size(0)
            val_correct += (val_pred_cls == val_target_cls).sum().item()

    val_loss = loss_fn(y_batch_val, val_logits)  # 計算最後批次的損失值

    # 印出每個迭代回合的驗證評估結果
    print('Training loss over epoch: %.4f' % (float(loss_value.item()),))
    val_acc = val_correct / val_samples  # 平均所有批次的評估結果
    print('Validation acc: %.4f' % (float(val_acc),))

    # 將驗證的評估結果儲存下來
    val_acc_list.append(val_acc)
    val_loss_list.append(val_loss.cpu().detach().numpy())

    print('Time taken: %.2fs' % (time.time() - start_time))

<a name="EvaluateModel"></a>
## 評估模型

* ### 視覺化訓練過程的評估指標 （Visualization）

In [ ]:
plt.figure(figsize=(15, 4))
plt.subplot(1, 2, 1)
plt.plot(range(len(train_loss_list)), train_loss_list, label='train_loss')
plt.plot(range(len(val_loss_list)), val_loss_list, label='valid_loss')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.legend()

plt.subplot(1, 2, 2)
plt.plot(range(len(train_acc_list)), train_acc_list, label='train_acc')
plt.plot(range(len(val_acc_list)), val_acc_list, label='valid_acc')
plt.xlabel('Epochs')
plt.ylabel('Accuracy')
plt.legend()
plt.show()

* ### 模型預測（Model predictions）

In [ ]:
val_pred = []
for x_val, y_val in val_dataset:
    if torch.cuda.is_available():
        x_val = x_val.cuda()
        y_val = y_val.cuda()
    val_pred += list(model(x_val).argmax(-1).flatten().cpu())

In [ ]:
val_pred[:10]

In [ ]:
len(val_pred)

In [ ]:
from sklearn.metrics import classification_report
print(classification_report(y_valid.argmax(-1), val_pred))